In [1]:
type ∨[P, Q] = Either[P, Q]
type ∧[P, Q] = Tuple2[P, Q]
type ⟶[P, Q] = P => Q
type ⊥ = Nothing
type True = Unit
type ¬[P] = P => ⊥

defined type ∨
defined type ∧
defined type ⟶
defined type ⊥
defined type True
defined type ¬

In [2]:
type Or[P, Q] = Either[P, Q]
type And[P, Q] = Tuple2[P, Q]
type Implies[P, Q] = P => Q
type False = Nothing
type True = Unit
type Not[P] = P => ⊥

defined type Or
defined type And
defined type Implies
defined type False
defined type True
defined type Not

In [3]:
class Inhabitant{ x => 
    // Knight(x) -- `x` is a Knight
    // 
    type Knight
    
    // Knave(x) -- `x` is a Knave (i.e. is not a Knight)
    // 
    type Knave = ¬[Knight]
    
    // Says(x, P) -- `x` says that `P` holds, i.e. asserts proposition `P`
    // 
    type Says[P]
}

defined class Inhabitant

In [4]:
trait KnightsKnaves{
    // P1. Inhabitants are knights or knaves
    // 
    //     ∀ x. Inhabitant(x) ⟶ Knight(x) ∨ Knave(x) 
    // 
    def P1(x: Inhabitant): x.Knight ∨ x.Knave
    
    // In Scala 3
    // val P1: (x: Inhabitant) => Either[x.Knight, x.Knave]
    
    // P2. Knights are truth tellers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ P
    // 
    def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
    
    // In Scala 3
    // val P2: [P] => (x: Inhabitant) => x.Knight => x.Says[P] => P
    
    // P3. Knaves are persistent liers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ ¬P
    // 
    def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
}

defined trait KnightsKnaves

# Puzzle 1 - Ejemplo.
Is it possible for any inhabitant of this island to claim that he is a knave?


No; no inhabitant can claim to be a knave because no knight would lie and say he is a knave and no knave would truthfully admit to being a knave.

In [5]:
// { P1, P2, P3 } ⊢ ∀ x. Inhabitant(x) ⟶ ¬Says(x, Knave(x))

def puzzle1(premises: KnightsKnaves)(x: Inhabitant): ¬[x.Says[x.Knave]] = 
    // 1. Says(x, Knave(x))                                       ; hypothesis
    ((_1: x.Says[x.Knave]) => 
         (premises.P1(x) match {
             // 2. Knight(x)                                      ; hypothesis
             case Left(_2: x.Knight) => 
                 //   3. Knight(x) → Says(x, Knave(x)) → Knave(x) ; P2[Knave(x),x]
                 val _3: x.Knight => x.Says[x.Knave] => x.Knave = premises.P2[x.Knave](x)
                 //   4. Says(x, Knave(x)) → Knave(x)             ; ⟶E(3,2)
                 val _4: x.Says[x.Knave] => x.Knave = _3(_2)
                 //   5. Knave(x)                                 ; ⟶E(4,1)
                 //   5. ¬ Knight(x)                              ; ≝ Knave
                 //   5. Knight(x) → ⊥                            ; ≝ ¬ 
                 val _5: x.Knight => ⊥ = _4(_1)
                 //   6. ⊥                                        ; ⟶E(5,2)
                 _5(_2) : ⊥

             //  _7. Knave(x)                                     ; hypothesis
             case Right(_7: x.Knave) => 
                 //   8. Knave(x) → Says(x, Knave(x)) → ¬ Knave(x) ; P3[Knave(x),x]
                 val _8: x.Knave => x.Says[x.Knave] => ¬[x.Knave] = premises.P3[x.Knave](x)
                 //   9. Says(x, Knave(x)) → ¬ Knave(x)            ; ⟶E(8,7)
                 val _9: x.Says[x.Knave] => ¬[x.Knave] = _8(_7)
                 //   10. ¬ Knave(x)                               ; ⟶E(9,1)
                 //   10. Knave(x) → ⊥                             ; ≝ ¬ 
                 val _10: x.Knave => ⊥ = _9(_1)
                 //  11. ⊥                                         ; ⟶E(10,7)
                 _10(_7) : ⊥

        // _12. ⊥                                                  ; ∨E(P1, 2-6, 7-11)
        }) : False

    // _13. Says(x, Knave(x)) ⟶ ⊥                                  ; ⟶I(1-12)
    // _13. ¬ Says(x, Knave(x))                                    ; ≝ ¬ 
    ) : ¬[x.Says[x.Knave]]

defined function puzzle1

In [6]:
// { P1, P2, P3 } ⊢ ∀ x. Inhabitant(x) ⟶ ¬Says(x, Knave(x))

def puzzle1(premises: KnightsKnaves)(x: Inhabitant): x.Says[x.Knave] => Nothing =
    // 1. Says(x, Knave(x))                                       ; hypothesis
    ((_1: x.Says[x.Knave]) => 
         (premises.P1(x) match {
             // 2. Knight(x)                                      ; hypothesis
             case Left(_2: x.Knight) => 
                 //   3. Knight(x) → Says(x, Knave(x)) → Knave(x) ; P2[Knave(x),x]
                 val _3: x.Knight => x.Says[x.Knave] => x.Knave = premises.P2[x.Knave](x)
                 //   4. Says(x, Knave(x)) → Knave(x)             ; ⟶E(3,2)
                 val _4: x.Says[x.Knave] => x.Knave = _3(_2)
                 //   5. Knave(x)                                 ; ⟶E(4,1)
                 //   5. ¬ Knight(x)                              ; ≝ Knave
                 //   5. Knight(x) → ⊥                            ; ≝ ¬ 
                 val _5: x.Knight => ⊥ = _4(_1)
                 //   6. ⊥                                        ; ⟶E(5,2)
                 _5(_2) : ⊥

             //  _7. Knave(x)                                     ; hypothesis
             case Right(_7: x.Knave) => 
                 //   8. Knave(x) → Says(x, Knave(x)) → ¬ Knave(x) ; P3[Knave(x),x]
                 val _8: x.Knave => x.Says[x.Knave] => ¬[x.Knave] = premises.P3[x.Knave](x)
                 //   9. Says(x, Knave(x)) → ¬ Knave(x)            ; ⟶E(8,7)
                 val _9: x.Says[x.Knave] => ¬[x.Knave] = _8(_7)
                 //   10. ¬ Knave(x)                               ; ⟶E(9,1)
                 //   10. Knave(x) → ⊥                             ; ≝ ¬ 
                 val _10: x.Knave => ⊥ = _9(_1)
                 //  11. ⊥                                         ; ⟶E(10,7)
                 _10(_7) : ⊥

        // _12. ⊥                                                  ; ∨E(P1, 2-6, 7-11)
        }) : False

    // _13. Says(x, Knave(x)) ⟶ ⊥                                  ; ⟶I(1-12)
    // _13. ¬ Says(x, Knave(x))                                    ; ≝ ¬ 
    ) : ¬[x.Says[x.Knave]]

defined function puzzle1

In [7]:
implicit class SomeSugar(P: KnightsKnaves){    
    def eitherKnightOrKnave(x: Inhabitant): x.Knight Or x.Knave = P.P1(x)
    def knightsAreTruthTellers[P](x: Inhabitant) = P.P2[P](x)
    def knavesAreLiers[P](x: Inhabitant) = P.P3[P](x)
    
    def noKnightLies[P](x: Inhabitant): x.Knight => x.Says[Not[P]] => P => False = 
        xIsKnight => xSaysNotP => p => 
            P.P2[Not[P]](x)(xIsKnight)(xSaysNotP)(p)
    
    def noKnaveTellsTruth[P](x: Inhabitant): x.Knave => x.Says[P] => P => False = 
        xIsKnave => xSaysP => p => 
            P.P3[P](x)(xIsKnave)(xSaysP)(p)
}

defined class SomeSugar

In [8]:
// { P1, P2, P3 } ⊢ ∀ x. Inhabitant(x) ⟶ ¬ Says(x, Knave(x))

def puzzle1(premises: KnightsKnaves)(x: Inhabitant): Not[x.Says[x.Knave]] =
    xSaysIsKnave =>
        premises.eitherKnightOrKnave(x).fold(
            xIsKnight => 
                // no knight would lie
                premises.noKnightLies(x)(xIsKnight)(xSaysIsKnave)(xIsKnight),
            xIsKnave => 
                // no knave would tell the truth
                premises.noKnaveTellsTruth(x)(xIsKnave)(xSaysIsKnave)(xIsKnave)
        )

defined function puzzle1

# Puzzle 26 - Knight and Knave

In [8]:
/* Según este viejo problema, tres de los habitantes - A, B, C - se encontraban en un jardin.
Un extranjero pasó por allí y le preguntó a A, "¿Eres caballero o escudero?".
A respondió, pero tan confusamente, que el extranjero no pudo enterarse de lo que decía.
Entonces el extranjero preguntó a B, "¿Que ha dicho A?". Y B le respondió "A ha dicho que es escudero".
Pero en ese instante el tercer hombre, C, dijo "¡No creas a B, que está mintiendo!"

Pregunta: ¿que son B y C?

Opcion1: B - escudero
         C - escudero
SOLUCION CORRECTA ---------------->  Opcion2: B - escudero
                                              C - caballero
Opcion3: B - caballero
         C - escudero
Opcion4: B - caballero
         C - caballero  
*/         

In [8]:
/* Detalle de la Solucion:

Es imposible que un caballero o un escudero digan: "Yo soy escudero", porque un caballero emitiría el enunciado
falso de que él es escudero, y un escudero no emitiría el enunciado verdadero de que él es un escudero.
Por lo tanto, A nunca diría que era un escudero. Así, B mentía cuando dijo que A habia dicho que él era escudero.
Por lo tanto B es escudero. Puesto que C dijo que B estaba mintiendo, y B estaba ciertamente mintiendo, C dijo la verdad, 
de aquí que se tratara de un caballero. Asi pues, B es un escudero y C un caballero. (Es imposible saber que es A).

*/

In [8]:
/* Definicion en LPO:

A - 1
B - 2
C - 3

Says(P[2], Says(P[1], P[1] = Knave)) ∧ Says(P[3], P[2] = Knave)
B: A says he is a knave ---> Says(P[2], Says(P[1], P[1] = Knave))
C: B is a knave         ---> Says(P[3], P[2] = Knave)

Pseudo ejemplo:

Says(B, Says(A, knave(A))) ∧ Says(C, knave(B)) => (knave(B) ∧ knight(C))

*/

In [8]:
/*def puzzle26(premises: KnightsKnaves)(a: Inhabitant)(b: Inhabitant)(c: Inhabitant): 
    (b.Says[a.Says[a.knave]] And c.Says[b.knave]) => (b.knave And c.knave)


*/

In [10]:
//NUEVA VERSION
//DEFINICION INCORRECTA

def puzzle26(premises: KnightsKnaves)(a: Inhabitant)(b: Inhabitant)(c: Inhabitant):
    b.Says[(a.Says[a.Knave], c.Says[b.Knave])] => (b.Knave, c.Knight) = 
    ({bSay: b.Says[(a.Says[a.Knave], c.Says[b.Knave])] => 
        (premises.P1(b) match{
            case Left(bKnight: b.Knight) => 
                //def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                val b_1L: b.Knight => 
                            b.Says[(a.Says[a.Knave], c.Says[b.Knave])] => 
                                (a.Says[a.Knave], c.Says[b.Knave]) = premises.P2[(a.Says[a.Knave], c.Says[b.Knave])](b)
                val b_2L: b.Says[(a.Says[a.Knave], c.Says[b.Knave])] => 
                                (a.Says[a.Knave], c.Says[b.Knave]) = b_1L(bKnight)
                val b_3L: (a.Says[a.Knave], c.Says[b.Knave]) = b_2L(bSay)
                b_3L._1: a.Says[a.Knave] 
                b_3L._2: c.Says[b.Knave]
                
                (premises.P1(c) match{
                    case Left(cKnight: c.Knight) =>
                        //def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                        val c_1L: c.Knight => c.Says[b.Knave] => b.Knave = premises.P2[b.Knave](c)
                        val c_2L: c.Says[b.Knave] => b.Knave = c_1L(cKnight)
                        val c_3L: b.Knave = c_2L(b_3L._2)
                                                       
                        (c_3L: b.Knave, cKnight : c.Knight) : (b.Knave, c.Knight)
                    //¿Seria necesaria esta rama Right si ya tengo la solcuion en Left?
                    case Right(cKnave: c.Knave) => 
                        //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                        val c_1R: c.Knave => c.Says[b.Knave] => Not[b.Knave] = premises.P3[b.Knave](c)
                        val c_2R: c.Says[b.Knave] => Not[b.Knave] = c_1R(cKnave)
                        //val c_3R: Not[b.Knave] = c_2R(csayR)
                        val c_3R: b.Knave => Nothing = c_2R(b_3L._2)
                        
                        c_4R: b.Knave => 
                            c_3R(c_4R) : Nothing
                            
                        (c_3R(c_4R), c_3R(c_4R)) : (b.Knave, c.Knight)
                        // ¿Porqué teniendo ya algo de tipo (b.Knave, c.Knight) no puedo quitar las ultimas ??? ?
                        // ¿No obtengo solucion en esta rama porque solo es correcta si b es un knave?
                })
            ??? : (b.Knave, c.Knight)
            
            case Right(bKnave: b.Knave) => 
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val b_1R: b.Knave => 
                            b.Says[(a.Says[a.Knave], c.Says[b.Knave])] => 
                                Not[(a.Says[a.Knave], c.Says[b.Knave])] = premises.P3[(a.Says[a.Knave], c.Says[b.Knave])](b)
                val b_2R: b.Says[(a.Says[a.Knave], c.Says[b.Knave])] => 
                                Not[(a.Says[a.Knave], c.Says[b.Knave])] = b_1R(bKnave)
                //val b_3R: Not[(a.Says[a.Knave], c.Says[b.Knave])] = b_2R(bsayR)
                val b_3R: ((a.Says[a.Knave], c.Says[b.Knave])) => Nothing = b_2R(bSay) 
                
                b_4R: ((a.Says[a.Knave], c.Says[b.Knave])) =>
                    b_3R(b_4R): Nothing
            
            (bKnave, b_3R(b_4R)) : (b.Knave, c.Knight)
        })
    ??? : (b.Knave, c.Knight)})

defined function puzzle26

In [ ]:
//DEFINICION CORRECTA PERO HAY QUE CORREGIR EL PARAMETRO DE ENTRADA QUE ES UNA TUPLA (( )) de ahi el error

In [11]:
def puzzle26(premises: KnightsKnaves)(a: Inhabitant)(b: Inhabitant)(c: Inhabitant):
    (b.Says[a.Says[a.Knave]], c.Says[b.Knave]) => (b.Knave, c.Knight) = ???

defined function puzzle26

In [12]:
def puzzle26(premises: KnightsKnaves)(a: Inhabitant)(b: Inhabitant)(c: Inhabitant):
    ((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) => (b.Knave, c.Knight) = 
    
    ??? : (((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) => (b.Knave, c.Knight))
    

defined function puzzle26

In [12]:
def puzzle26(premises: KnightsKnaves)(a: Inhabitant)(b: Inhabitant)(c: Inhabitant):
    (b.Says[a.Says[a.Knave]], c.Says[b.Knave]) => (b.Knave, c.Knight) = 
    ({t: (b.Says[a.Says[a.Knave]], c.Says[b.Knave]) =>
        
        ??? : (b.Knave, c.Knight)
        
    }): ((b.Says[a.Says[a.Knave]], c.Says[b.Knave]) => (b.Knave, c.Knight))
        
    
    

cmd12.sc:3: type mismatch;
 found   : ((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) => (b.Knave, c.Knight)
    (which expands to)  ((b.Says[a.Says[a.Knight => Nothing]], c.Says[b.Knight => Nothing])) => (b.Knight => Nothing, c.Knight)
 required: (b.Says[a.Says[a.Knave]], c.Says[b.Knave]) => (b.Knave, c.Knight)
    (which expands to)  (b.Says[a.Says[a.Knight => Nothing]], c.Says[b.Knight => Nothing]) => (b.Knight => Nothing, c.Knight)
    ({t: (b.Says[a.Says[a.Knave]], c.Says[b.Knave]) =>
                                                    ^Compilation Failed

: 

In [17]:
//DEFINICION CORRECTA CORREGIDA
def puzzle26(premises: KnightsKnaves)(a: Inhabitant)(b: Inhabitant)(c: Inhabitant):
    ((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) => (b.Knave, c.Knight) = 
    ({t: ((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) =>
        t._1: b.Says[a.Says[a.Knave]]
        t._2: c.Says[b.Knave]
        
        premises.P1(c) match{
            case Left(cKnight: c.Knight) =>
            
                ??? : (b.Knave, c.Knight)
            
            case Right(cKnave: c.Knave) =>
            
                ??? : (b.Knave, c.Knight)
        }
         
        
        ??? : (b.Knave, c.Knight)
        
    }): (((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) => (b.Knave, c.Knight))
        
    
    

defined function puzzle26

In [25]:
def puzzle26(premises: KnightsKnaves)(a: Inhabitant)(b: Inhabitant)(c: Inhabitant):
    ((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) => (b.Knave, c.Knight) = 
    ({t: ((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) =>
        t._1: b.Says[a.Says[a.Knave]]
        t._2: c.Says[b.Knave]
        
        premises.P1(c) match{
            case Left(cKnight: c.Knight) =>
                // def P2[P](c: Inhabitant): c.Knight => c.Says[P] => P     
                val c_L1: c.Knight => c.Says[b.Knave] => b.Knave = premises.P2[b.Knave](c)
                val c_L2: c.Says[b.Knave] => b.Knave = c_L1(cKnight)
                val c_L3: b.Knave = c_L2(t._2)
            
                (c_L2(t._2): b.Knave, cKnight: c.Knight)
            
            case Right(cKnave: c.Knave) =>
                //def P3[P](c: Inhabitant): c.Knave => c.Says[P] => ¬[P]
                val c_R1: c.Knave => c.Says[b.Knave] => ¬[b.Knave] = premises.P3[b.Knave](c)
                val c_R2: c.Says[b.Knave] => ¬[b.Knave] = c_R1(cKnave)
                val c_R3: b.Knave => Nothing = c_R2(t._2)
            
                ((premises.P1(b) match{
                    case Right(bKnave: b.Knave) =>
                        bKnave: b.Knave
                }): b.Knave, ??? : c.Knight)
        }
         
        
        ??? : (b.Knave, c.Knight)
        
    }): (((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) => (b.Knave, c.Knight))
        
    
    

defined function puzzle26

In [29]:
def puzzle26(premises: KnightsKnaves)(a: Inhabitant)(b: Inhabitant)(c: Inhabitant):
    ((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) => (b.Knave, c.Knight) = 
    ({t: ((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) =>
        t._1: b.Says[a.Says[a.Knave]]
        t._2: c.Says[b.Knave]
        
        (premises.P1(c) match{
            case Left(cKnight: c.Knight) =>
                // def P2[P](c: Inhabitant): c.Knight => c.Says[P] => P     
                val c_L1: c.Knight => c.Says[b.Knave] => b.Knave = premises.P2[b.Knave](c)
                val c_L2: c.Says[b.Knave] => b.Knave = c_L1(cKnight)
                val c_L3: b.Knave = c_L2(t._2)
            
                (c_L2(t._2): b.Knave, cKnight: c.Knight)
            
            case Right(cKnave: c.Knave) =>
                //def P3[P](c: Inhabitant): c.Knave => c.Says[P] => ¬[P]
                val c_R1: c.Knave => c.Says[b.Knave] => ¬[b.Knave] = premises.P3[b.Knave](c)
                val c_R2: c.Says[b.Knave] => ¬[b.Knave] = c_R1(cKnave)
                val c_R3: b.Knave => Nothing = c_R2(t._2)
                
                ((premises.P1(b) match{
                    case Right(bKnave: b.Knave) =>
                        bKnave: b.Knave
                }): b.Knave, c_R3(premises.P1(b) match{
                    case Right(bKnave: b.Knave) =>
                        bKnave: b.Knave
                }) : c.Knight)
        }) : (b.Knave, c.Knight)
        
    }): (((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) => (b.Knave, c.Knight))
        
    
    

defined function puzzle26